In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import df_stored

In [ ]:
# Assuming you want to use the dataframe from df_stored
#match = df_stored.match_df
match_df = pd.read_csv('../data/raw/Match.csv')
match_original_df = pd.read_csv('../data/raw/Match.csv')
match_df.head()

In [ ]:
match_df = match_df.drop(columns=['PSH', 'PSD', 'PSA'], errors='ignore')
match_df.head()

In [ ]:
# Display non-empty (non-null) values from the 'goal' column
non_empty_goals = match_df['goal'].dropna()
print(non_empty_goals.iloc[0])

In [ ]:
team_df = pd.read_csv('../data/raw/Team.csv')
team_df.head()

In [ ]:
# Merge team_df to match_df to get home and away team long names
match_df = match_df.merge(
    team_df[['team_api_id', 'team_long_name']],
    left_on='home_team_api_id',
    right_on='team_api_id',
    how='left'
).rename(columns={'team_long_name': 'home_team_long_name'}).drop('team_api_id', axis=1)

match_df = match_df.merge(
    team_df[['team_api_id', 'team_long_name']],
    left_on='away_team_api_id',
    right_on='team_api_id',
    how='left'
).rename(columns={'team_long_name': 'away_team_long_name'}).drop('team_api_id', axis=1)

# Show the first few rows with new columns
match_df[['home_team_api_id', 'home_team_long_name', 'away_team_api_id', 'away_team_long_name']].head()

In [ ]:
print(match_df.columns[11])

In [ ]:
match_df.head()

In [ ]:
match_df = match_df.drop(match_df.columns[11:55], axis=1)
match_df.head()

In [ ]:
home_player_cols = [f'home_player_{i}' for i in range(1, 12)]
missing_counts = match_df[home_player_cols].isnull().sum()
missing_percent = match_df[home_player_cols].isnull().mean() * 100

stats_df = pd.DataFrame({
    'missing_count': missing_counts,
    'missing_percent': missing_percent
})

print(stats_df)

In [ ]:
match_df.head()

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(match_df[home_player_cols].isnull(), cbar=False, cmap='viridis', yticklabels=False)
plt.title('Missing Values Heatmap for Home Players 1-11')
plt.xlabel('Home Player Columns')
plt.show()

In [ ]:
match_df = match_df.dropna(subset=['home_team_long_name', 'away_team_long_name'])


In [ ]:
print(f"Number of rows in match_df: {len(match_df)}")


In [ ]:
player_df = pd.read_csv('../data/cleaned/player_attributes_cleaned.csv')
print(player_df.head())

In [ ]:
# ...existing code...

# Prepare mapping from player_api_id to player_name and overall_rating
# Keep the latest entry for each player_api_id (or you can use .max()/.min() as needed)
player_info = player_df[['player_api_id', 'player_name', 'overall_rating']].drop_duplicates(subset=['player_api_id'], keep='last')

# Create mapping dictionaries with unique indices
player_name_map = player_info.set_index('player_api_id')['player_name']
player_rating_map = player_info.set_index('player_api_id')['overall_rating']

# Define player columns
home_player_cols = [f'home_player_{i}' for i in range(1, 12)]
away_player_cols = [f'away_player_{i}' for i in range(1, 12)]
all_player_cols = home_player_cols + away_player_cols

# Add player name and rating columns using .map()
for col in all_player_cols:
    match_df[f'{col}_name'] = match_df[col].map(player_name_map)
    match_df[f'{col}_overall'] = match_df[col].map(player_rating_map)

# Show the first few rows with new columns
cols_to_show = []
for col in all_player_cols:
    cols_to_show.extend([col, f'{col}_name', f'{col}_overall'])
#match_df[cols_to_show].tail()

# ...existing code...

# Remove duplicate columns by column name, keeping the last occurrence
match_df = match_df.loc[:, ~match_df.columns.duplicated(keep='last')]

# ...existing code...
match_df[cols_to_show].tail()

In [ ]:
# ...existing code...

# List of league IDs to keep
league_ids_to_keep = [1729, 4769, 7809, 10257, 21518]

# Number of rows before filtering
rows_before = len(match_df)

# Filter the DataFrame
match_df = match_df[match_df['league_id'].isin(league_ids_to_keep)]

# Number of rows after filtering
rows_after = len(match_df)

print(f"Rows before filtering: {rows_before}")
print(f"Rows after filtering: {rows_after}")

#

In [ ]:
# Count rows with any missing data
missing_rows = match_df.isnull().any(axis=1).sum()
print(f"Number of rows with missing data: {missing_rows}")


In [ ]:
# Remove specified columns if they exist
cols_to_remove = ['goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession']
match_df = match_df.drop(columns=cols_to_remove, errors='ignore')
match_df.head()


In [ ]:
# Move all columns from 'B365H' to 'BSA' to the end of the DataFrame
betting_start = match_df.columns.get_loc('B365H')
betting_end = match_df.columns.get_loc('BSA') + 1  # +1 to include 'BSA'

betting_cols = match_df.columns[betting_start:betting_end].tolist()
other_cols = [col for col in match_df.columns if col not in betting_cols]

# Reorder columns: all non-betting columns first, then betting columns
match_df = match_df[other_cols + betting_cols]
match_df.head()

In [ ]:
# Print total number of rows
total_rows = len(match_df)
print(f"Total number of rows: {total_rows}")

# Calculate and print percent of rows with missing data
missing_rows = match_df.isnull().any(axis=1).sum()
percent_missing = (missing_rows / total_rows) * 100
print(f"Percent of rows with missing data: {percent_missing:.2f}%")
match_df 

In [ ]:
# Get the range of betting columns
betting_start = match_df.columns.get_loc('B365H')
betting_end = match_df.columns.get_loc('BSA') + 1  # +1 to include 'BSA'
betting_cols = match_df.columns[betting_start:betting_end].tolist()

# All columns except betting columns
non_betting_cols = [col for col in match_df.columns if col not in betting_cols]

# Count rows with missing data in non-betting columns
missing_non_betting = match_df[non_betting_cols].isnull().any(axis=1).sum()
print(f"Rows with missing data (excluding betting columns): {missing_non_betting}")

In [ ]:
# Remove Italy (league_id=10257) and France (league_id=4769) leagues
match_df = match_df[~match_df['league_id'].isin([10257, 4769])]

# Get the range of betting columns
betting_start = match_df.columns.get_loc('B365H')
betting_end = match_df.columns.get_loc('BSA') + 1  # +1 to include 'BSA'
betting_cols = match_df.columns[betting_start:betting_end].tolist()

# All columns except betting columns
non_betting_cols = [col for col in match_df.columns if col not in betting_cols]

# Count rows with missing data in non-betting columns
missing_non_betting = match_df[non_betting_cols].isnull().any(axis=1).sum()
print(f"Rows with missing data (excluding betting columns, after removing Italy and France): {missing_non_betting}")
# Print total number of rows after removing Italy and France
print(f"Total number of rows:   {len(match_df)}")

In [ ]:
# Get the exact betting columns from 'B365H' to 'BSA'
betting_start = match_df.columns.get_loc('B365H')
betting_end = match_df.columns.get_loc('BSA') + 1  # +1 to include 'BSA'
betting_cols = match_df.columns[betting_start:betting_end].tolist()

# Home, draw, and away columns within the betting odds range only
home_cols = [col for col in betting_cols if col.endswith('H')]
draw_cols = [col for col in betting_cols if col.endswith('D')]
away_cols = [col for col in betting_cols if col.endswith('A')]

# Calculate the median for each row for home, draw, and away odds (using only B365H to BSA columns)
match_df['betting_median_home'] = match_df[home_cols].median(axis=1, skipna=True)
match_df['betting_median_draw'] = match_df[draw_cols].median(axis=1, skipna=True)
match_df['betting_median_away'] = match_df[away_cols].median(axis=1, skipna=True)

# Calculate probabilities based on the median odds
inv_home = 1 / match_df['betting_median_home']
inv_draw = 1 / match_df['betting_median_draw']
inv_away = 1 / match_df['betting_median_away']
inv_sum = inv_home + inv_draw + inv_away

match_df['prob_home'] = inv_home / inv_sum
match_df['prob_draw'] = inv_draw / inv_sum
match_df['prob_away'] = inv_away

match_df.head()

In [ ]:
# Define player columns for home and away (1-11)
home_player_cols = [f'home_player_{i}' for i in range(1, 12)]
away_player_cols = [f'away_player_{i}' for i in range(1, 12)]
all_player_cols = home_player_cols + away_player_cols

# Total number of player slots (rows * 22)
total_player_slots = len(match_df) * len(all_player_cols)

# Total number of missing player slots
missing_players = match_df[all_player_cols].isnull().sum().sum()

# Percentage of missing player slots
percent_missing_players = (missing_players / total_player_slots) * 100
print(f"Percentage of missing player slots: {percent_missing_players:.2f}%")

In [ ]:
# Drop all rows with any missing player slots (home_player_1-11 and away_player_1-11)
home_player_cols = [f'home_player_{i}' for i in range(1, 12)]
away_player_cols = [f'away_player_{i}' for i in range(1, 12)]
all_player_cols = home_player_cols + away_player_cols

match_df = match_df.dropna(subset=all_player_cols)
print(f"Rows remaining after dropping missing player slots: {len(match_df)}")

In [ ]:
# Calculate percent of rows with missing data before column 'B365H'
b365h_idx = match_df.columns.get_loc('B365H')
cols_before_b365h = match_df.columns[:b365h_idx]

missing_rows = match_df[cols_before_b365h].isnull().any(axis=1).sum()
percent_missing = (missing_rows / len(match_df)) * 100
print(f"Percent of rows with missing data before 'B365H': {percent_missing:.2f}%")

In [ ]:
# Show which columns before 'B365H' have missing data and how many missing values per column
b365h_idx = match_df.columns.get_loc('B365H')
cols_before_b365h = match_df.columns[:b365h_idx]

missing_counts = match_df[cols_before_b365h].isnull().sum()
missing_cols = missing_counts[missing_counts > 0]
print("Columns before 'B365H' with missing data and their counts:")
print(missing_cols)

In [ ]:
match_df.head

In [ ]:
match_df.to_csv('../data/cleaned/match_df_cleaned.csv', index=False)